In [1]:
# set up the big query connector
# must be on python3
# Ref: https://cloud.google.com/bigquery/docs/visualize-jupyter
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery df
select
  federal_action_obligation
  , total_dollars_obligated
from `ian-sandbox-221014.ian_usaspending.transactions`
limit 1000

In [4]:
# running the big query jupyter magic dumps the data into a pandas dataframe!
type(df)

pandas.core.frame.DataFrame

In [ ]:
# feature/target engineering
# create hold out set

In [ ]:
# cross validation
  # feature selection, dimensionality reduction, etc.
  # model training
  # hyperparameter optimization
  # trained model evaluation
  # model selection

In [ ]:
# final model evaluation against hold out set

In [ ]:
# deploy model 

In [ ]:
# score new data against deployed model